In [13]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [14]:
df = pd.read_excel('data.xlsx')
df.head()

,YEAR,RANK,TEAM,MANAGER,W,L,T,CLINCHED_PLAYOFFS,PTS_FOR,PTS_AGNST,WAIVER,MOVES,STREAK
0,2021,1,Your Coach,Marsh,7,7,0,Y,1841.36,1790.30,7,31,W-2
1,2021,2,#Trash_Commish,Masi,9,5,0,Y,1978.36,1656.44,10,24,W-1
2,2021,3,Golden Rams Fam,Shoey,10,4,0,Y,1824.06,1799.68,8,15,W-2
3,2021,4,Shmo Stars,Zach,6,8,0,Y,1767.56,1727.72,9,26,W-4
4,2021,5,AngelLikeMoniBoloni,Moni,9,5,0,Y,1841.04,1760.78,2,17,W-1


# Average rank

In [15]:
df_average_rank = df.copy()
df_average_rank = df_average_rank.groupby(['MANAGER'])['RANK'].mean().to_frame().reset_index()
with open('../src/data/average_rank.json', 'w') as fout:
    json.dump(df_average_rank.to_dict(orient='list'), fout, indent=2)

In [16]:
fig = px.bar(df_average_rank, x='MANAGER', y='RANK')
fig.update_layout({
    'showlegend': False,
    'xaxis': {
        'title': 'Manager'
    },
    'yaxis': {
        'title': 'Average Rank'
    }
})

# Most top 3 finishes

In [17]:
df_placed = df.copy()
df_placed = df_placed[df_placed['RANK'] <= 3]
df_placed = df_placed.groupby(['MANAGER', 'RANK']).size().unstack(fill_value=0).reset_index()
df_placed = df_placed.rename(columns={1: '1st', 2: '2nd', 3: '3rd'})
with open('../src/data/top_three_finishes.json', 'w') as fout:
    json.dump(df_placed.to_dict(orient='list'), fout, indent=2)

In [18]:
fig = px.bar(df_placed, x='MANAGER', y=['1st', '2nd', '3rd'])
fig.update_layout({
    'yaxis': {
        'title': 'Rank'
    }, 
    'legend': {
        'title': ''
    }
})

fig.show()